In [18]:
!pip install evaluate
!pip install swifter

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.2 MB 812.7 kB/s eta 0:00:02
     ----------------- ---------------------- 0.5/1.2 MB 4.7 MB/s eta 0:00:01
     ---------------------------------------  1.2/1.2 MB 7.4 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16519 sha256=b8507ebf802b336d491958945b88e22033286f55eafe1eec654277515baf5b80
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\ef\7f\bd\9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


In [2]:
import pandas as pd
import numpy as np
import os
import random
import evaluate
from tqdm import tqdm

In [3]:
TRAIN_PATH = '/mnt/elice/dataset/train/'
TEST_PATH = '/mnt/elice/dataset/test/'

In [6]:
df = pd.read_csv(TRAIN_PATH + 'texts.csv')
raw_submission = pd.read_csv('raw_submission.csv')

In [7]:
def remove_duplicates(s):
    l = s.split(" ")
    while len(l) >= 2 and l[-1] == l[-2]:
        l = l[:-1]
    return " ".join(l)

In [8]:
print(remove_duplicates("같이 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진"))
print(remove_duplicates("아 아 아 아 아"))

같이 사진
아


In [13]:
raw_submission["text"] = raw_submission["text"].apply(remove_duplicates)

In [14]:
import re

def clean_text(text, remove_space=True):
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\]','', text)
    if remove_space:
        text = ''.join(text.split())
    return text

In [16]:
targets = list(map(lambda x : clean_text(x), set(df['text'].tolist()))) #train label값
threshold = 41 # text길이
cer_threshold = 0.5
metric = evaluate.load('cer')
def get_close(pred):
    if len(pred) >= threshold:
        return pred
    min_cer = 1e8
    index = -1
    for j in range(len(targets)):
        target = targets[j]
        if len(target) >= threshold:
            continue
        if len(target) > len(pred) * 2:
            continue
        if len(pred) > len(target) * 1.5:
            continue 
        cnt = 0
        for ch in target:
            if ch in pred:
                cnt += 1
        if cnt * 2 < len(target):
            continue
        cer = metric.compute(predictions=[pred], references=[target])
        if min_cer > cer:
            min_cer = cer
            index = j
    if min_cer < cer_threshold:
        pred = targets[index]
    return pred

In [19]:
import swifter
targets = list(map(lambda x : clean_text(x), set(df['text'].tolist()))) #train label값
preds = raw_submission['text'].apply(lambda x : clean_text(x))# predict 값

tqdm.pandas()
preds = preds.swifter.progress_bar(True).allow_dask_on_strings(enable=True).apply(get_close)
    
    

In [72]:
submission = raw_submission
submission['text'] = preds

In [73]:
submission.to_csv('submission.csv', index=False)